<a href="https://colab.research.google.com/github/mjiii25/22-1_DLAI/blob/main/gilbut_ch7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter 7

## Functional API

In [1]:
from keras import Input, layers

input_tensor = Input(shape = (32,))
dense = layers.Dense(32, activation = 'relu')

output_tensor = dense(input_tensor)

Sequential model vs Functinal API

In [2]:
from keras.models import Sequential, Model
from keras import layers
from keras import Input

## Sequential model
seq_model = Sequential()
seq_model.add(layers.Dense(32, activation = 'relu', input_shape = (64,)))
seq_model.add(layers.Dense(32, activation = 'relu'))
seq_model.add(layers.Dense(10, activation = 'softmax'))


## Functional API
input_tensor = Input(shape = (64,))
x = layers.Dense(32, activation = 'relu')(input_tensor)
x = layers.Dense(32, activation = 'relu')(x)
output_tensor = layers.Dense(10, activation = 'softmax')(x)

model = Model(input_tensor, output_tensor)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 64)]              0         
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 32)                1056      
                                                                 
 dense_6 (Dense)             (None, 10)                330       
                                                                 
Total params: 3,466
Trainable params: 3,466
Non-trainable params: 0
_________________________________________________________________


In [3]:
model.compile(optimizer = 'rmsprop',
              loss = 'categorical_crossentropy')

import numpy as np
x_train = np.random.random((1000,64))
y_train = np.random.random((1000,10))

model.fit(x_train, y_train, epochs = 10, batch_size = 128)
score = model.evaluate(x_train, y_train)

Epoch 1/10
8/8 [==============================] - 3s 4ms/step - loss: 11.9352
Epoch 2/10
8/8 [==============================] - 0s 4ms/step - loss: 12.9515
Epoch 3/10
8/8 [==============================] - 0s 4ms/step - loss: 15.1002
Epoch 4/10
8/8 [==============================] - 0s 4ms/step - loss: 18.2318
Epoch 5/10
8/8 [==============================] - 0s 4ms/step - loss: 22.3672
Epoch 6/10
8/8 [==============================] - 0s 4ms/step - loss: 26.8313
Epoch 7/10
8/8 [==============================] - 0s 4ms/step - loss: 30.6468
Epoch 8/10
8/8 [==============================] - 0s 4ms/step - loss: 34.1296
Epoch 9/10
8/8 [==============================] - 0s 4ms/step - loss: 37.5523
Epoch 10/10
32/32 [==============================] - 0s 3ms/step - loss: 40.7666


In [4]:
score

40.76655197143555

Multi-input model

In [5]:
from keras.models import Model
from keras import layers
from keras import Input


text_vocabulary_size = 10000
question_vocabulary_size = 10000
answer_vocabulary_size = 500

## Text
text_input = Input(shape = (None,), dtype = 'int32', name = 'text')
embedded_text = layers.Embedding(text_vocabulary_size, 64)(text_input)
encoded_text = layers.LSTM(32)(embedded_text)

## Question
question_input = Input(shape = (None,), dtype = 'int32', name = 'question')
embedded_question = layers.Embedding(question_vocabulary_size, 32)(question_input)
encoded_question = layers.LSTM(16)(embedded_question)

concatenated = layers.concatenate([encoded_text, encoded_question], axis = -1)    # connect encoded text and question

answer = layers.Dense(answer_vocabulary_size, activation = 'softmax')(concatenated)

model = Model([text_input, question_input], answer)
model.compile(optimizer = 'rmsprop', loss = 'categorical_crossentropy', metrics = ['acc'])

In [ ]:
import numpy as np
from keras.utils import to_categorical

num_samples = 1000
max_length = 100

text = np.random.randint(1, text_vocabulary_size, size = (num_samples, max_length))
question = np.random.randint(1, question_vocaulary_size, size = (num_samples, max_length))

answers = np.random.randint(0, answer_vocabulary_size, size = num_samples)
answers = to_categorical(answers)                                                 # one-hot encoded

model.fit([text, question], answers, epochs = 10, batch_size = 128)
model.fit({'text' : text, 'question' : question}, answers, epochs = 10, batch_size = 128)

ImportError: ignored

multi-output model

In [6]:
from keras import layers
from keras import Input
from keras.models import Model
vocabulary_size = 50000
num_income_groups = 10

posts_input = Input(shape = (None,), dtype = 'int32', name = 'posts')
embedded_input = layers.Embedding(vocabulary_size, 256)(posts_input)
x = layers.Conv1D(128, 5, activation = 'relu')(embedded_input)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation = 'relu')(x)
x = layers.Conv1D(256, 5, activation = 'relu')(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation = 'relu')(x)
x = layers.Conv1D(256, 5, activation = 'relu')(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation = 'relu')(x)

age_prediction = layers.Dense(1, name = 'age')(x)
income_prediction = layers.Dense(num_income_groups, activation = 'softmax', name = 'income')(x)
gender_prediction = layers.Dense(1, activation = 'sigmoid', name = 'gender')(x)

model = Model(posts_input, [age_prediction, income_prediction, gender_prediction])

In [ ]:
model.compile(optimizer = 'rmsprop',
              loss = ['mse', 'categorical_crossentropy', 'binary_crossentropy'],
              loss_weights = [0.25, 1., 10.])

## or

model.compile(optimizer = 'rmsprop',
              loss = {'age' : 'mse',                                              # scalar regression
                      'income' : 'categorical_crossentropy',
                      'gender' : 'binary_crossentropy'})                          # binary classification

Inception Module

In [ ]:
from keras import layers
branch_a = 